In [1]:
from datetime import datetime
from my_stuff import BybitTestKeys
from my_stuff import MufexKeys
from my_stuff import MufexKeys, MufexTestKeys
from plotly.subplots import make_subplots
from quantfreedom.enums import *
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.helper_funcs import candles_to_df
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plotting_base import *
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.ex_strat_03 import SMACrossing
from quantfreedom.strategies.ex_strat_004 import RSIRisingFalling
import numpy as np
import numpy as np
import pandas as pd


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=3000,
)
timestamps = candles[:, CandleBodyType.Timestamp]
datetimes = timestamps.astype("datetime64[ms]")
open = candles[:, CandleBodyType.Open]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]
volume = candles[:, CandleBodyType.Volume]

In [3]:
short_strat = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.array([65]),
)
short_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
short_strat.plot_signals(candles=candles)

In [6]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.array([35]),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
long_strat.plot_signals(candles=candles)

In [ ]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([0]),
    sl_based_on_add_pct=np.array([0, 0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="market",
    tp_strategy_type=TakeProfitStrategyType.Provided,
    trail_sl_bool=False,
    z_or_e_type=None,
)